In [ ]:
import os
import sys
import pandas as pd
import yaml 
from matplotlib import pyplot as plt
from matplotlib import ticker as mticker
import statsmodels.api as sm
import numpy as np
from itertools import product
import subprocess

with open("../../config.yaml.local", "r") as f:
    LOCAL_CONFIG = yaml.safe_load(f)
#with open("../../config.yaml", "r") as f:
#    CONFIG = yaml.safe_load(f)
sys.path.append("../python")

import globals
import data_tools as dt

LOCAL_PATH = LOCAL_CONFIG["LOCAL_PATH"]
RAW_DATA_PATH = LOCAL_CONFIG["RAW_DATA_PATH"]
DATA_PATH = LOCAL_CONFIG["DATA_PATH"]
R_PATH = LOCAL_CONFIG["R_PATH"]

RUN_R_SCRIPTS = False
OVERWRITE = False


In [ ]:
items = dt.get_items(overwrite=OVERWRITE)
posts = dt.get_posts(overwrite=OVERWRITE)
comments = dt.get_comments(overwrite=OVERWRITE)

In [7]:
posts['downzappers48'].describe()

count    259409.000000
mean          0.042439
std           0.253816
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          11.000000
Name: downzappers48, dtype: float64